In [21]:
# import all the dependencies
import tensorflow as tf 
import numpy as np
import librosa, librosa.display
from librosa.util import fix_length
import scipy as sp
# !pip install spafe==0.2.0
import spafe # make sure that the version of spafe installed is spafe==0.2.0
from scipy.stats import kurtosis,skew,mode,gstd,describe,iqr,gmean,hmean,median_abs_deviation,variation,tstd,gstd,moment,entropy
from spafe.features.gfcc import gfcc
from sklearn.preprocessing import normalize

model=tf.keras.models.load_model('/content/drive/MyDrive/Data/acoustic_classifier_mvd') # insert path of trained model here

path='/content/drive/MyDrive/Eusipco Paper/Omar-Audios-All/Motorcycle/Recording_1001_O_M.wav'  #Insert path of audio fetched from the app

y, sr = librosa.load(path)
required_audio_size = 3
padded_signal = fix_length(y, size=required_audio_size*sr) 

# code for matrix
Matrix  = gfcc(sig=padded_signal, fs=sr, num_ceps=40,nfilts=128,nfft=2048,win_hop=0.0232,win_len=0.0464)
Matrix=np.reshape(Matrix,[1,128,40])

# code for vector
ft=sp.fft.fft(y)
magnitude=np.absolute(ft)
hist=magnitude[0:11025]
k=kurtosis(hist)
s=skew(hist)
mean=np.mean(hist)
z=np.array(mode(hist)[0])
mode_var=float(z)
i=iqr(hist)
g=gmean(hist)
h=hmean(hist)
dev=median_abs_deviation(hist)
var=variation(hist)
variance=np.var(hist)
std=tstd(hist)
gstd_var=gstd(hist)
ent= entropy(hist)
    

features=[mode_var,k,s,mean,i,g,h,dev,var,variance,std,gstd_var,ent]
vector=np.array(features)
vector=normalize([vector])
vector=np.reshape(vector,[1,13])

# code for prediction

predict=model.predict([vector,Matrix]) 
predict1=(predict > 0.5).astype("int32")
confidence=predict

if predict1[0][0]==1:
  print("It is a car")
  print("confidence is",confidence[0][0]*100 ,"%" )
elif predict1[0][1]==1:
  print("It is a motorbike")
  print("confidence is",confidence[0][1]*100,"%" )
elif predict1[0][3]==1:
  print("It is a truck")
  print("confidence is",confidence[0][3]*100,"%" )
else:
  print("There is no vehicle in the audio")

# {'C': 0, 'M': 1, 'N': 2, 'T': 3}



1/1 [==============================] - 0s 136ms/step
It is a motorbike
confidence is 94.07369494438171 %
